<a href="https://colab.research.google.com/github/Anerol18/Fake_News_Detector_NLP_DeepLearning_Project/blob/main/User_Interface_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#load the saved model
!wget https://github.com/Anerol18/Fake_News_Detector_NLP_DeepLearning_Project/raw/master/stella_model.pth


--2024-10-24 07:49:02--  https://github.com/Anerol18/Fake_News_Detector_NLP_DeepLearning_Project/raw/master/stella_model.pth
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/Anerol18/Fake_News_Detector_NLP_DeepLearning_Project/master/stella_model.pth [following]
--2024-10-24 07:49:02--  https://media.githubusercontent.com/media/Anerol18/Fake_News_Detector_NLP_DeepLearning_Project/master/stella_model.pth
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66097468 (63M) [application/octet-stream]
Saving to: ‘stella_model.pth’

stella_model.pth    100%[===================>]  63.04M 

In [1]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel

# Load the embedding model (dunzhang/stella_en_1.5B_v5)
model_name = "dunzhang/stella_en_1.5B_v5"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 20.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user


In [2]:
# Load the tokenizer and the pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
embedding_model = AutoModel.from_pretrained(model_name, trust_remote_code=True).to(device).eval()

tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

tokenization_qwen.py:   0%|          | 0.00/10.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/dunzhang/stella_en_1.5B_v5:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

modeling_qwen.py:   0%|          | 0.00/65.2k [00:00<?, ?B/s]

ImportError: This modeling file requires the following packages that were not found in your environment: flash_attn. Run `pip install flash_attn`

In [2]:
# Define the SimpleNN class, with same achitecture used for the stella_model
class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        # The input size for fc0 should match the saved model's input size
        self.fc0 = nn.Linear(input_size, 3072)
        self.dropout0 = nn.Dropout(p=0.6)
        self.relu0 = nn.ReLU()
        self.fc01 = nn.Linear(3072, 3072)
        self.dropout01 = nn.Dropout(p=0.6)
        self.relu01 = nn.ReLU()
        self.fc1 = nn.Linear(3072, 768)
        self.dropout1 = nn.Dropout(p=0.6)
        self.relu1 = nn.ReLU()
        self.fc3 = nn.Linear(768, 2)

    def forward(self, x):
        x = self.fc0(x)
        x = self.dropout0(x)
        x = self.relu0(x)
        x = self.fc01(x)
        x = self.dropout01(x)
        x = self.relu01(x)
        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.relu1(x)
        x = self.fc3(x)
        return x

# Load the model
# Change input_size to 1536 to match the saved model's first layer input
input_size = 1536
model = SimpleNN(input_size)

# Load the model weights
model_path = 'stella_model.pth'  # Path to your model file
model.load_state_dict(torch.load(model_path))

# Set the model to evaluation

<ipython-input-2-daf41d32b88c>:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [3]:
def preprocess_input(sentence, tokenizer):
    # Tokenize the input sentence using the tokenizer from your model
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
    input_tensor = inputs['input_ids'].to(device)
    return input_tensor

In [4]:
# Define a function to make predictions
def predict_real_or_fake(sentence, model, tokenizer):
    # Preprocess the input sentence
    input_tensor = preprocess_input(sentence, tokenizer)

    # Pass the tokenized input through the model
    with torch.no_grad():
        output = model(input_tensor)
        _, predicted = torch.max(output, 1)

    # Map prediction to label
    if predicted.item() == 0:
        return "Real"
    else:
        return "Fake"


In [6]:
# Get user input
user_input = input("Enter a sentence: ")
prediction = predict_real_or_fake(user_input, model, tokenizer)  # Make the prediction
print(f"The prediction is: {prediction}")  # Display the result


Enter a sentence: Covid is a virus


NameError: name 'tokenizer' is not defined

In [5]:
# Get user input
user_input = input("Enter a sentence: ")
prediction = predict(user_input)  # Make the prediction
print(f"The prediction is: {prediction}")  # Display the result

KeyboardInterrupt: Interrupted by user